Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [138]:
from random import choices,randint

import lab9_lib

In [139]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11110111010100010110010000111000010100010101100111: 15.33%
10010000111101010100101010010001100100101011011101: 7.33%
01000110111111001111111100101100100000001101111001: 23.33%
00110100101110011001011100011000101010110010101111: 9.13%
10111000111011100101011010111111110100101001111000: 31.34%
10101000101011101011000011010011001100010001000011: 7.33%
01010111110010100100011100100011100001001101111110: 15.33%
01110110000010101000110000100011010101110101010011: 7.33%
01110100001000110011011111110100010011100011001111: 7.33%
11011010010000110011001100111001000000000001011111: 23.56%
10


## EA

In [140]:
import random

def mutation(genome):
    index = randint(0,len(genome[0])-1)
    genome[0][index] = 1-genome[0][index]
    return genome[0]

def one_cut_xover(ind1, ind2):
    cut_point = randint(0, len(ind1[0]))
    offspring = ind1[0][:cut_point]+ind2[0][cut_point:]
    return offspring

def xover(genome1, genome2):
    child_genome = [g1 if random.random() > 0.5 else g2 for g1, g2 in zip(genome1[0], genome2[0])]
    return child_genome

In [141]:
from random import choice

def select_parent(population,fitness):
    best_parents = sorted(population,key= lambda i:i[1],reverse=True)[:int(len(population)/2)]
    pool = [choice(best_parents) for _ in range(int(len(best_parents)/10))]
    champion = max(pool, key=lambda i: i[1])
    return champion

In [142]:
def init_population(n_individual,length,fitness):
    pop = []
    for _ in range(n_individual):
        ind = (choices([0, 1], k=length))
        pop.append((ind,fitness(ind)))
    return pop

In [143]:
def gen_new_population(n_new_indivdual,mutation_prob,old_population,fitness):
    new_individual = []
    for _ in range(n_new_indivdual):
        if random.random() < mutation_prob:
            old_ind = select_parent(old_population,fitness)
            tmp = mutation(old_ind)
        else:
            old_ind = select_parent(old_population,fitness)
            old_ind_2 = select_parent(old_population,fitness)
            tmp = xover(old_ind,old_ind_2)
        new_individual.append((tmp,fitness(tmp)))
    return new_individual

In [144]:
def replacement(new_pop,old_pop,fitness):
    tmp_pop = new_pop + old_pop
    sorted_pop = sorted(tmp_pop,key= lambda i:i[1],reverse=True)
    return sorted_pop[:len(new_pop)]

In [145]:
N_INDV = 100
LENGTH_INDV = 1000
GENERATION = 300

problem_size = [1,2,5,10]

for ps in problem_size:
    fit = lab9_lib.make_problem(ps)
    pop = init_population(N_INDV,LENGTH_INDV,fit)
    best = 0
    n_calls = 0
    gen = 0
    for g in range(GENERATION):
        #if g%10 == 0:
        #    avg_fit = sum(list(map(lambda i:i[1],pop)))/len(pop)
        #    print(pop[0][1], avg_fit)
        new_pop = gen_new_population(N_INDV,0.1,pop,fit)
        pop = replacement(new_pop,pop,fit)
        if pop[0][1] > best:
            best = pop[0][1]
            n_calls = fit.calls
            gen = g
    print(f"Problem size: {ps}")
    print(f"Best fitness: {best}")
    print(f"Fitness calls: {n_calls}")
    print(f"Generation: {gen}")
    print(f"Population size: {N_INDV}")

Problem size: 1
Best fitness: 0.982
Fitness calls: 29700
Generation: 295
Population size: 100
Problem size: 2
Best fitness: 0.682
Fitness calls: 30100
Generation: 299
Population size: 100
Problem size: 5
Best fitness: 0.34658
Fitness calls: 30100
Generation: 299
Population size: 100
Problem size: 10
Best fitness: 0.218122235
Fitness calls: 30100
Generation: 299
Population size: 100
